In [1]:
import pandas as pd

file_path = "evaluate_v2.xlsx"
labels = []
checks = []

for i in range(58,60+1):
    for j in range(0,2+1):
        sheet_name = f"result_{i}_try{j}"
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        labels.extend(df['라벨 (1: 첫 번 째 후보군이 더 나은 경우, 2 : 두 번 째 후보군이 더 나은 경우, 3: 둘 다 괜찮은 경우. 4 : 둘 다 별로인 경우)'])
        checks.extend(df['오역여부'])

In [2]:
print(len(labels), len(checks))

849 849


In [3]:
#오역을 개선한 경우
b_cnt = 0#성능 개선 개수
w_cnt = 0#성능 저하 개수
critical_b_cnt = 0#오역이었다가 개선된 개수
critical_w_cnt = 0#오역으로 악화된 개수
total_worse_cnt = 0#오역인 경우의 개수
total_normal_cnt = 0 #오역이 아니었던 경우의 개수
total_cnt = len(labels)

for label, check in zip(labels, checks):
    if label == 2:
        b_cnt += 1
        if check == 'O':
            critical_b_cnt += 1
            total_worse_cnt += 1
        else:
            total_normal_cnt += 1#1은 오역이 아님
    elif label == 1:
        w_cnt += 1
        if check == 'O':
            total_worse_cnt += 1#2에서 오역이 생긴 경우
            critical_w_cnt += 1
        else:
            total_normal_cnt += 1#1은 오역이 아니라는 거
    elif label == 4:
        if check == 'O':
            total_worse_cnt += 1
        else:
            total_normal_cnt += 1
    else:#label==3, 1>3
        total_normal_cnt += 1 #오역이 아님

#원래 오역이었던 경우 + 오역이 생긴 경우        
print(f"[처리 전]총 오역 수 : {total_worse_cnt}건/{total_cnt}건, 비율 : {round(total_worse_cnt/total_cnt*100,2)}%")
#원래 오역이었는데 -> 2&오역 o
print(f"오역을 개선한 경우 : {critical_b_cnt}건/{total_worse_cnt-critical_w_cnt}건, 비율 : {round(critical_b_cnt/(total_worse_cnt-critical_w_cnt)*100,2)}%") 
#원래 오역이었는데 -> 계속 4로 라벨링
print(f"오역을 개선하지 못한 경우 : {(total_worse_cnt-critical_w_cnt-critical_b_cnt)}건/{total_worse_cnt-critical_w_cnt}건, 비율 : {round((total_worse_cnt-critical_w_cnt-critical_b_cnt)/(total_worse_cnt-critical_w_cnt)*100,2)}%")
#오역이 아니었는데(1,2&오역x,3) -> 오역이 된 경우 : 1&오역O(후보군 2가 오역인 경우) 
print(f"오역이 추가로 발생한 경우 : {critical_w_cnt}건/{total_normal_cnt}건, 비율 : {round((critical_w_cnt)/total_normal_cnt*100,2)}%")
print(f"[처리 후]총 오역 수: {total_worse_cnt-critical_b_cnt}건/{total_cnt}건, 비율 : {round((total_worse_cnt-critical_b_cnt)/total_cnt*100,2)}%")
print()
print(f"번역 선호도가 상승한 경우 : {b_cnt}건/{total_cnt}건, 비율 : {round(b_cnt/total_cnt*100,2)}%")
print(f"번역 선호도가 하락한 경우 : {w_cnt}건/{total_cnt}건, 비율 : {round(w_cnt/total_cnt*100,2)}%")
print(f"번역 선호도가 그대로인 경우 : {total_cnt-w_cnt-b_cnt}건/{total_cnt}건, 비율 : {round((total_cnt-w_cnt-b_cnt)/total_cnt*100,2)}%")

[처리 전]총 오역 수 : 17건/849건, 비율 : 2.0%
오역을 개선한 경우 : 0건/17건, 비율 : 0.0%
오역을 개선하지 못한 경우 : 17건/17건, 비율 : 100.0%
오역이 추가로 발생한 경우 : 0건/832건, 비율 : 0.0%
[처리 후]총 오역 수: 17건/849건, 비율 : 2.0%

번역 선호도가 상승한 경우 : 100건/849건, 비율 : 11.78%
번역 선호도가 하락한 경우 : 183건/849건, 비율 : 21.55%
번역 선호도가 그대로인 경우 : 566건/849건, 비율 : 66.67%


In [25]:
sheet_name = f"관용어구가 포함된 평가 시트"
df_idiom = pd.read_excel(file_path, sheet_name=sheet_name)
print(df_idiom.keys())

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5'],
      dtype='object')


In [50]:
idiom_b_cnt = 0
idiom_w_cnt = 0
idiom_total_cnt = len(df_idiom)
critical_b_cnt = 0
critical_w_cnt = 0
total_w_cnt = 0
case_4_cnt = 0
for label, check in zip(df_idiom['Unnamed: 4'][1:], df_idiom['Unnamed: 5'][1:]):
    #print(label, check)
    if label == 1:
        idiom_w_cnt +=1
        if check == 'O':
            critical_w_cnt += 1 #오역 생성
    elif label == 2:
        idiom_b_cnt += 1
        if check == 'O':
            critical_b_cnt += 1 #개선
            total_w_cnt += 1
    elif label == 4:
        total_w_cnt += 1
        case_4_cnt += 1

print(f"[적용 전]오역 수 : {case_4_cnt}건/{idiom_total_cnt}건, 비율 : {round(case_4_cnt/idiom_total_cnt*100,2)}%")
print(f"오역을 개선한 경우 : {critical_b_cnt}건/{total_w_cnt}건, 비율 : {round(critical_b_cnt/total_w_cnt*100,2)}%")
print(f"오역이 추가로 생성된 경우 : {critical_w_cnt}건/{idiom_total_cnt-total_w_cnt}건, 비율 : {round(critical_w_cnt/(idiom_total_cnt-total_w_cnt),2)}%")
print(f"[적용 후]오역 수 : {case_4_cnt-critical_b_cnt+critical_w_cnt}건/{idiom_total_cnt}건, 비율 : {round(case_4_cnt-critical_b_cnt+critical_w_cnt/idiom_total_cnt*100,2)}%")
print()
print(f"번역 선호도가 상승한 경우 : {idiom_b_cnt}건/{idiom_total_cnt}건, 비율 : {round(idiom_b_cnt/idiom_total_cnt*100,2)}%")
print(f"번역 선호도가 하락한 경우 : {idiom_w_cnt}건/{idiom_total_cnt}건, 비율 : {round(idiom_w_cnt/idiom_total_cnt*100,2)}%")
print(f"번역 선호도가 그대로인 경우 : {idiom_total_cnt-idiom_w_cnt-idiom_b_cnt}건/{idiom_total_cnt}건, 비율 : {round((idiom_total_cnt-idiom_w_cnt-idiom_b_cnt)/idiom_total_cnt*100,2)}%")

[적용 전]오역 수 : 6건/117건, 비율 : 5.13%
오역을 개선한 경우 : 1건/7건, 비율 : 14.29%
오역이 추가로 생성된 경우 : 0건/110건, 비율 : 0.0%
[적용 후]오역 수 : 5건/117건, 비율 : 5.0%

번역 선호도가 상승한 경우 : 38건/117건, 비율 : 32.48%
번역 선호도가 하락한 경우 : 33건/117건, 비율 : 28.21%
번역 선호도가 그대로인 경우 : 46건/117건, 비율 : 39.32%
